In [11]:
%matplotlib ipympl
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import numpy as np
import scipy.io as sp
from mpl_toolkits.mplot3d import Axes3D

In [12]:
#Loading Dataset
dataSet = sp.loadmat('breastw.mat')
#Splitting dataset
Xtraining = dataSet['X'] #in this first run we use the whole dataset
X = Xtraining
#Labels
y = dataSet['y']
y.shape
reduce2D = PCA(n_components = 2)
Xreduced2D = reduce2D.fit_transform(X)

Xbenign2D =  Xreduced2D[np.where(y==0)[0]]
Xmalign2D =  Xreduced2D[np.where(y==1)[0]]

In [17]:
mat = np.empty((0,50))
Counter = -1
for Init in ["k-means++", "random"]:
    for N_init in range(1,50,1):
        for Max_iter in range(100,900):
            for Algorithm in ["full", "elkan"]:
                Counter += 1
                
                model = KMeans(n_clusters = 2, init = Init, n_init = N_init, max_iter = Max_iter, algorithm=Algorithm)
                model.fit(Xtraining)
                centers = model.cluster_centers_
                kmeansEstimate = model.predict(X)

                #Correcting labels
                reversedEstimate = abs(kmeansEstimate-1)

                DefaultComparison  = np.reshape(kmeansEstimate,(-1,1))-y
                ReversedComparison = np.reshape(reversedEstimate,(-1,1))-y

                Def = np.count_nonzero(DefaultComparison == 0)
                rev = np.count_nonzero(ReversedComparison == 0)

                if rev>Def:
                    kmeansEstimate = reversedEstimate

                XbenignEst2D =  Xreduced2D[np.where(kmeansEstimate==0)[0]]
                XmalignEst2D =  Xreduced2D[np.where(kmeansEstimate==1)[0]]

                modelSumUp = [Init, N_init,Max_iter,Algorithm, kmeansEstimate]
                np.vstack([mat, modelSumUp])
                print(Counter)


"""
#Creating model
model = KMeans(n_clusters = 2)
#Training model i.e determining cluster centers
model.fit(Xtraining)
centers = model.cluster_centers_
centersreduced2D = reduce2D.fit_transform(centers)
#Applying model to dataSet
kmeansEstimate = model.predict(X)
#Switching cluster labels to match with dataset labels
#kmeansEstimate *= -1
#kmeansEstimate +=1
"""


C:\Users\Pro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 50 and the array at index 1 has size 5